# Day 5: Supply Stacks - Part One

The expedition can depart as soon as the final supplies have been unloaded from the ships. Supplies are stored in stacks of marked crates, but because the needed supplies are buried under many other crates, the crates need to be rearranged.

The ship has a giant cargo crane capable of moving crates between stacks. To ensure none of the crates get crushed or fall over, the crane operator will rearrange them in a series of carefully-planned steps. After the crates are rearranged, the desired crates will be at the top of each stack.

The Elves don't want to interrupt the crane operator during this delicate procedure, but they forgot to ask her which crate will end up where, and they want to be ready to unload them as soon as possible so they can embark.

They do, however, have a drawing of the starting stacks of crates and the rearrangement procedure (your puzzle input). For example:
```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
```

In this example, there are three stacks of crates. Stack 1 contains two crates: crate Z is on the bottom, and crate N is on top. Stack 2 contains three crates; from bottom to top, they are crates M, C, and D. Finally, stack 3 contains a single crate, P.

Then, the rearrangement procedure is given. In each step of the procedure, a quantity of crates is moved from one stack to a different stack. In the first step of the above rearrangement procedure, one crate is moved from stack 2 to stack 1, resulting in this configuration:
```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```

In the second step, three crates are moved from stack 1 to stack 3. Crates are moved one at a time, so the first crate to be moved (D) ends up below the second and third crates:

```
        [Z]
        [N]
    [C] [D]
    [M] [P]
 1   2   3
```

Then, both crates are moved from stack 2 to stack 1. Again, because crates are moved one at a time, crate C ends up below crate M:

```
        [Z]
        [N]
[M]     [D]
[C]     [P]
 1   2   3
```

Finally, one crate is moved from stack 1 to stack 2:
```
        [Z]
        [N]
        [D]
[C] [M] [P]
 1   2   3
```
The Elves just need to know which crate will end up on top of each stack; in this example, the top crates are C in stack 1, M in stack 2, and Z in stack 3, so you should combine these together and give the Elves the message CMZ.

**After the rearrangement procedure completes, what crate ends up on top of each stack?**

In [1]:
import numpy as np
import pandas as pd

In [2]:
crate_directions = pd.read_csv('data/day-5-camp-cleanup-directions.csv')
crate_directions['n_crates'] = crate_directions.directions.str.split(' ').str[1]
crate_directions['_from'] = crate_directions.directions.str.split(' ').str[3]
crate_directions['_to'] = crate_directions.directions.str.split(' ').str[5]

In [3]:
crate_directions.head(10)

,directions,n_crates,_from,_to
0,move 1 from 8 to 4,1,8,4
1,move 1 from 7 to 8,1,7,8
2,move 1 from 6 to 3,1,6,3
3,move 2 from 6 to 5,2,6,5
4,move 8 from 5 to 1,8,5,1
5,move 5 from 3 to 8,5,3,8
6,move 1 from 7 to 8,1,7,8
7,move 8 from 1 to 2,8,1,2
8,move 3 from 3 to 9,3,3,9
9,move 13 from 8 to 7,13,8,7


In [4]:
crates = pd.read_csv('data/day-5-camp-cleanup-crates.csv')
d = pd.concat([pd.DataFrame([np.NaN] * crates.notna().sum().sum() ), crates]).drop(columns=0).reset_index(drop=True)

In [5]:
d.tail(20)

,1,2,3,4,5,6,7,8,9
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def get_first_nan_index_of_column(d: pd.DataFrame, column: str) -> int:
    return max(d[d[column].isna()].index)

def get_first_non_nan_index_of_column(d: pd.DataFrame, column: str) -> int:
    return min(d[~d[column].isna()].index)

def shuffling(d: pd.DataFrame, n_crates: str, _from: str, _to:str):
    for i in d[_from].dropna().values[:int(n_crates)]:
        m = get_first_nan_index_of_column(d, _to)
        n = get_first_non_nan_index_of_column(d, _from)
        d.iloc[m, int(_to) - 1] = i
        d.iloc[n, int(_from) - 1] = np.NaN
    

In [7]:
def rearragement_process(d: pd.DataFrame, crate_directions: pd.DataFrame) -> pd.DataFrame:
    for i, row in crate_directions.iterrows():
        shuffling(d, row['n_crates'], row['_from'], row['_to'])
    return d
    

In [8]:
df = d.copy()
data = rearragement_process(df, crate_directions)
data.tail(20)

,1,2,3,4,5,6,7,8,9
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
combine = ''.join([data[field].dropna().reset_index(drop=True).tolist()[0] for field in data.columns])
f'The combination of crates that end up on top of each stack is: {combine}'


'The combination of crates that end up on top of each stack is: SBPQRSCDF'

# Part Two

As you watch the crane operator expertly rearrange the crates, you notice the process isn't following your prediction.

Some mud was covering the writing on the side of the crane, and you quickly wipe it away. The crane isn't a `CrateMover 9000` - it's a `CrateMover 9001`.

The CrateMover 9001 is notable for many new and exciting features: air conditioning, leather seats, an extra cup holder, and the ability to pick up and move multiple crates at once.

Again considering the example above, the crates begin in the same configuration:
```
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
Moving a single crate from stack 2 to stack 1 behaves the same as before:
```
[D]        
[N] [C]    
[Z] [M] [P]
 1   2   3 
```
However, the action of moving three crates from stack 1 to stack 3 means that those three moved crates stay in the same order, resulting in this new configuration:
```
        [D]
        [N]
    [C] [Z]
    [M] [P]
 1   2   3
```
Next, as both crates are moved from stack 2 to stack 1, they retain their order as well:
```
        [D]
        [N]
[C]     [Z]
[M]     [P]
 1   2   3
```
Finally, a single crate is still moved from stack 1 to stack 2, but now it's crate C that gets moved:
```
        [D]
        [N]
        [Z]
[M] [C] [P]
 1   2   3
```
In this example, the CrateMover 9001 has put the crates in a totally different order: MCD.

**Before the rearrangement process finishes, update your simulation so that the Elves know where they should stand to be ready to unload the final supplies. After the rearrangement procedure completes, what crate ends up on top of each stack?**

In [11]:
def new_shuffling(d, n_crates, _from, _to):
    list_of_values = d[_from].dropna().values[:int(n_crates)] if n_crates == '1' else list(d[_from].dropna().values)[:int(n_crates)][::-1]
    for i in list_of_values:
        m = get_first_nan_index_of_column(d, _to)
        n = get_first_non_nan_index_of_column(d, _from)
        d.iloc[m, int(_to) - 1] = i
        d.iloc[n, int(_from) - 1] = np.NaN

In [12]:
def new_rearragement_process(d: pd.DataFrame, crate_directions: pd.DataFrame) -> pd.DataFrame:
    for i, row in crate_directions.iterrows():
        new_shuffling(d, row['n_crates'], row['_from'], row['_to'])
    return d
    

In [13]:
df = d.copy()
data = new_rearragement_process(df, crate_directions)
data.tail(20)

,1,2,3,4,5,6,7,8,9
44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
combine = ''.join([data[field].dropna().reset_index(drop=True).tolist()[0] for field in data.columns])
f'The new combination of crates that end up on top of each stack is: {combine}'


'The new combination of crates that end up on top of each stack is: RGLVRCQSB'